<a href="https://colab.research.google.com/github/jessicamotam/AvantiBootcamp_ML/blob/main/ML_Classificacao_Imagens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Etapa 1: importar dataset do Kaggle

In [8]:
#baixando datasets do kaggle e instalando o pandas
!pip install opendatasets
!pip install pandas

In [9]:
import opendatasets as od
import pandas

#importando dataset do COVID
od.download("https://www.kaggle.com/datasets/maedemaftouni/covid19-ct-scan-lesion-segmentation-dataset/data")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: jessicamotam
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/maedemaftouni/covid19-ct-scan-lesion-segmentation-dataset


100%|██████████| 688M/688M [00:09<00:00, 75.2MB/s]


In [10]:
# listando arquivos e pastas que tem no dataset
!ls

covid19-ct-scan-lesion-segmentation-dataset  sample_data


## Etapa 2: criando dataframe com os metadados da imagem

In [12]:
#importando biblioteca do OPEN CV(biblioteca para processamento de imagem e visão computacional)
  ## cv2 vai permitir tarefas de leitura e gravação de imagens, manipulação de imagens(redimensioamento, transformação e rotação), detecção de objetos, reconhecimento facial
import cv2

#biblioteca para acessar e manipular o sistema de arquivos
import os

In [19]:
# caminho para do diretório no sistema da base do covid / acessando a PASTA FRAMES

root_dir = './covid19-ct-scan-lesion-segmentation-dataset'

In [16]:
#intalando a biblioteca ImageHash -> usada para calculas hashes de imagens

!pip install ImageHash

In [27]:
#bibliotecas

import pandas as pd #manipulação de dados
import imagehash #calcula hashes de imagens(verifica similaridades ou detecta imagens corrompidas)
import matplotlib.pyplot as plt #visualização de graficos
%matplotlib inline
# exibe e gera os gráficos dentro do colab, na célula de baixo

from PIL import Image # importa a biblioteca PIL para manipulação de imagens
contar_imagens_corrompidas = 0
imagens_corrompidas = list() #armazena as imagens corrompidas

In [24]:
data_frame_list = list()

for folder in os.listdir(root_dir): #acessando as pastas do diretório
  child_dir = os.path.join(root_dir,folder)

  for image in os.listdir(child_dir): #acessando as imagens das pastas do diretorio
    img_dir = os.path.join(child_dir, image)
    _, image_format = image.split('.')
    img = cv2.imread(img_dir)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    if img is not None:
      #convertendo a imagem OPENCV para uma imagem do Pillow
      img_pill = Image.fromarray(img_rgb)

      #Gerando hash da imagem (usando percentual hash como exemplo)
      img_hash = imagehash.phash(img_pill)
      width, height, channels = img.shape
      img_corrupted = False

    else:
      contar_imagens_corrompidas += 1
      imagens_corrompidas(img_dir)
      img_corrompida = True
      img_hash, image_format, width, height, channels = None, None, None, None , None

      data_frame_list.append([img_dir, img_corrupted, img_hash, image_format,width, height, channels, folder]
                             )




## Mostrando se tem imagens corrompidas no DF

In [28]:
print(f'QUANTIDADE DE IMAGENS CORROMPIDAS: {contar_imagens_corrompidas} \n Imagens corrpompidas: {imagens_corrompidas}')


QUANTIDADE DE IMAGENS CORROMPIDAS: 0 
 Imagens corrpompidas: []


In [ ]:
df